In [1]:
# importing required classes
from pypdf import PdfReader, PdfWriter

In [2]:
# creating a pdf reader object
reader = PdfReader("source/composicoes.pdf")

# printing number of pages in pdf file
print((reader.pages))

[PageObject(0), PageObject(1), PageObject(2), PageObject(3), PageObject(4), PageObject(5), PageObject(6), PageObject(7), PageObject(8), PageObject(9), PageObject(10), PageObject(11), PageObject(12), PageObject(13), PageObject(14), PageObject(15), PageObject(16), PageObject(17), PageObject(18), PageObject(19), PageObject(20), PageObject(21), PageObject(22), PageObject(23), PageObject(24), PageObject(25), PageObject(26), PageObject(27), PageObject(28), PageObject(29), PageObject(30), PageObject(31), PageObject(32), PageObject(33), PageObject(34), PageObject(35), PageObject(36), PageObject(37), PageObject(38), PageObject(39), PageObject(40), PageObject(41), PageObject(42), PageObject(43), PageObject(44), PageObject(45), PageObject(46), PageObject(47), PageObject(48), PageObject(49), PageObject(50), PageObject(51), PageObject(52), PageObject(53), PageObject(54), PageObject(55), PageObject(56), PageObject(57), PageObject(58), PageObject(59), PageObject(60), PageObject(61), PageObject(62), P

In [3]:
import xlsxwriter, pandas as pd

In [4]:
composicoes = {
    "servico_title": [
        "servico",
        "descricao",
        "producao",
        "unidade",
        "custo unitario execucao",
        "custo_direto",
        "bdi",
        "preco_unitario_total",
        "pagina",
    ],
    "equipamentos_title": [
        "servico",
        "equipamento",
        "codigo",
        "ut. pr.",
        "ut. impr.",
        "vl. hr. prod.",
        "vl. hr. impr.",
        "consumo",
        "custo horario",
    ],
    "mao_de_obra_title": [
        "servico",
        "mao de obra",
        "codigo",
        "eq. salarial",
        "sal. hora",
        "consumo",
        "custo horario",
    ],
    "itens_de_incidencia_title": [
        "servico",
        "itens de incidencia",
        "codigo",
        "%",
        "M.O.",
        "equip.",
        "mat.",
        "custo",
    ],
    "materiais_title": [
        "servico",
        "materiais",
        "codigo",
        "unidade",
        "custo unitario",
        "consumo",
        "custo",
    ],
    "servicos_title": [
        "servico",
        "servicos",
        "codigo",
        "unidade",
        "custo unitario",
        "consumo",
        "custo",
    ],
    "itens_de_transporte_title": [
        "servico",
        "itens de transporte",
        "codigo",
        "unidade",
        "formula",
        "x1",
        "x2",
        "x3",
        "custo unitario",
        "consumo",
        "custo",
    ],
}

In [5]:
wb = xlsxwriter.Workbook("./target/servicos.xlsx")

In [6]:
def add_sheet(sheet_name, sheet_title):
    ws = wb.add_worksheet(sheet_name)
    for n, i in enumerate(sheet_title):
        ws.write(0, n, i)

In [7]:
for k, v in composicoes.items():
    print(k, v)
    add_sheet(k.replace("_title", "").replace("_", " "), v)

servico_title ['servico', 'descricao', 'producao', 'unidade', 'custo unitario execucao', 'custo_direto', 'bdi', 'preco_unitario_total', 'pagina']
equipamentos_title ['servico', 'equipamento', 'codigo', 'ut. pr.', 'ut. impr.', 'vl. hr. prod.', 'vl. hr. impr.', 'consumo', 'custo horario']
mao_de_obra_title ['servico', 'mao de obra', 'codigo', 'eq. salarial', 'sal. hora', 'consumo', 'custo horario']
itens_de_incidencia_title ['servico', 'itens de incidencia', 'codigo', '%', 'M.O.', 'equip.', 'mat.', 'custo']
materiais_title ['servico', 'materiais', 'codigo', 'unidade', 'custo unitario', 'consumo', 'custo']
servicos_title ['servico', 'servicos', 'codigo', 'unidade', 'custo unitario', 'consumo', 'custo']
itens_de_transporte_title ['servico', 'itens de transporte', 'codigo', 'unidade', 'formula', 'x1', 'x2', 'x3', 'custo unitario', 'consumo', 'custo']


In [8]:
global_servico = []
global_equipamentos = []
global_mao_de_obra = []
global_itens_de_incidencia = []
global_producao_equipe = []
global_materiais = []
global_servicos = []
global_itens_de_transporte = []
global_pages = {}
for n, page in enumerate(reader.pages):
    # extracting text from page
    texts = page.extract_text().splitlines()

    pagina = 4
    is_a = False
    is_b = False
    is_c = False
    is_not_c = False
    is_d = False
    is_e = False
    is_f = False
    is_g = False
    is_h = False
    is_not_h = False
    equipamento_s = []
    mao_de_obra_s = []
    itens_de_incidencia_s = []
    producao_equipe_s = []
    materiais_s = []
    servicos_s = []
    itens_de_transporte_s = []
    for text in texts:
        text_left = text.lstrip()
        if text_left.startswith("Serviço"):
            servico = text_left[
                text_left.find("Serviço")
                + len("Serviço")
                + 2 : text_left.find("Serviço")
                + len("Serviço")
                + 7
            ].strip()
            servico_nome = text_left[
                text_left.find("Serviço")
                + len("Serviço")
                + 9 : text_left.find("Unidade")
                - 1
            ].strip()
            unidade = text_left[
                text_left.find("Unidade") + len("Unidade") + 2 :
            ].strip()
            servico_page = {servico: n}
            global_pages.update(servico_page)
            # print(servico, unidade)
        if text_left.startswith("(A)"):
            is_a = True
        if text_left.startswith("(B)"):
            is_b = True
        if text_left.startswith("(C)"):
            is_c = True
        if text_left.startswith("Custo Horário"):
            is_not_c = True
        if text_left.startswith("(D)"):
            is_d = True
        if text_left.startswith("(E)"):
            is_e = True
        if text_left.startswith("(F)"):
            is_f = True
        if text_left.startswith("(G)"):
            is_g = True
        if text_left.startswith("(H)"):
            is_h = True
        if text_left.startswith("Custo Direto"):
            is_not_h = True
        if is_a == True and is_b == False:
            if not text_left.startswith("(A)"):
                _equipamento = text.split()
                equipamento = [
                    " ".join(_equipamento[: len(_equipamento) - 7])
                ] + _equipamento[-7:]
                equipamento_s.append(equipamento)
        if is_b == True and is_c == False:
            if not text_left.startswith("(B)"):
                _mao_de_obra = text.split()
                mao_de_obra = [
                    " ".join(_mao_de_obra[: len(_mao_de_obra) - 5])
                ] + _mao_de_obra[-5:]
                mao_de_obra_s.append(mao_de_obra)
        if is_c == True and is_not_c == False:
            if not text_left.startswith("(C)") and not text_left.startswith(
                "Custo Horário"
            ):
                _itens_de_incidencia = text.split()
                itens_de_incidencia = [
                    " ".join(_itens_de_incidencia[: len(_itens_de_incidencia) - 6])
                ] + _itens_de_incidencia[-6:]
                itens_de_incidencia_s.append(itens_de_incidencia)
        if is_f == True and is_g == False:
            if not text_left.startswith("(F)"):
                _materiais = text.split()
                materiais = [" ".join(_materiais[: len(_materiais) - 5])] + _materiais[
                    -5:
                ]
                materiais_s.append(materiais)
        if is_g == True and is_h == False:
            if not text_left.startswith("(G)"):
                _servicos = text.split()
                servicos = [" ".join(_servicos[: len(_servicos) - 5])] + _servicos[-5:]
                servicos_s.append(servicos)
        if is_h == True and is_not_h == False:
            if not text_left.startswith("(H)") and not text_left.startswith(
                "Custo Direto"
            ):
                _itens_de_transporte = text.split()
                itens_de_transporte = [
                    " ".join(_itens_de_transporte[: len(_itens_de_transporte) - 9])
                ] + _itens_de_transporte[-9:]
                itens_de_transporte_s.append(itens_de_transporte)
        if text_left.startswith("(D) Produção da Equipe"):
            producao = text_left[
                text_left.find("(D) Produção da Equipe")
                + len("(D) Produção da Equipe")
                + 1 :
            ].strip()
        if text_left.startswith("(E)"):
            custo_unitario_execucao = (
                text_left[text_left.find("(E)") + len("(E)") + 1 :].strip().split()[-1]
            )
        if text_left.startswith("Custo Direto"):
            custo_direto = (
                text_left[text_left.find("Custo Direto") + len("Custo Direto") + 1 :]
                .strip()
                .split()[-1]
            )
        if text_left.startswith("BDI"):
            bdi = (
                text_left[text_left.find("BDI") + len("BDI") + 1 :].strip().split()[-1]
            )
        if text_left.startswith("Preço Unitário"):
            preco_unitario_total = (
                text_left[
                    text_left.find("Preço Unitário") + len("Preço Unitário") + 1 :
                ]
                .strip()
                .split()[-1]
            )
            # unidade = text_left[text_left.find("Unidade") + len("Unidade") + 2 :]
            servicos = [
                servico,
                servico_nome,
                producao,
                unidade,
                custo_unitario_execucao,
                custo_direto,
                bdi,
                preco_unitario_total,
                n + 1,
            ]
    global_servico.append(servicos)
    for equipamento in equipamento_s:
        equipamento = [servico] + equipamento
        global_equipamentos.append(equipamento)
    for i in mao_de_obra_s:
        i = [servico] + i
        global_mao_de_obra.append(i)
    for i in itens_de_incidencia_s:
        i = [servico] + i
        global_itens_de_incidencia.append(i)
    for i in producao_equipe_s:
        i = [servico] + i
        global_producao_equipe.append(i)
    for i in materiais_s:
        i = [servico] + i
        global_materiais.append(i)
    for i in servicos_s:
        i = [servico] + i
        global_servicos.append(i)
    for i in itens_de_transporte_s:
        i = [servico] + i
        global_itens_de_transporte.append(i)

In [9]:
global_servico

[['40005',
  'Desmatamento,dest. e limpeza(arv.até 15cm)',
  '1,144.00',
  'm2',
  '0.15',
  '0.15',
  '0.04',
  '0.19',
  1],
 ['40010',
  'Desmatamento,dest. e limpeza(arv.D>15cm)',
  '1,394.00',
  'm2',
  '0.26',
  '0.26',
  '0.07',
  '0.33',
  2],
 ['40015',
  'Carga de entulhos',
  '191.00',
  'm3',
  '0.93',
  '0.93',
  '0.26',
  '1.19',
  3],
 ['40020',
  'Transporte de entulhos',
  '133.50',
  'm3km',
  '0.79',
  '0.79',
  '0.22',
  '1.01',
  4],
 ['40025',
  'Escavação e carga de material de 1ª cat',
  '220.00',
  'm3',
  '3.66',
  '3.66',
  '1.02',
  '4.68',
  5],
 ['40030',
  'Escavação e carga de material de 2ª cat.',
  '200.00',
  'm3',
  '5.31',
  '5.31',
  '1.48',
  '6.79',
  6],
 ['40045',
  'Escavação e carga de material de 3ª cat.',
  '40.00',
  'm3',
  '14.48',
  '27.62',
  '7.73',
  '35.35',
  7],
 ['40050',
  'Escavação e carga de solo mole',
  '23.00',
  'm3',
  '6.34',
  '6.34',
  '1.77',
  '8.11',
  8],
 ['40055',
  'Transporte mat.1°e2°cat(dt<=1km)',
  '78.00',

In [10]:
def add_data_sheet(sheet_name, sheet_vars):
    row = 1
    ws = wb.get_worksheet_by_name(sheet_name)
    for i in sheet_vars:
        for n, v in enumerate(i):
            ws.write(row, n, v)
        row += 1

In [11]:
for k, v in composicoes.items():
    add_data_sheet(
        k.replace("_title", "").replace("_", " "),
        globals()[f"global_{k.replace('_title', '')}"],
    )

In [16]:
wb.close()

In [17]:
global_pages

{'40005': 0,
 '40010': 1,
 '40015': 2,
 '40020': 3,
 '40025': 4,
 '40030': 5,
 '40045': 6,
 '40050': 7,
 '40055': 8,
 '40060': 9,
 '40065': 10,
 '40070': 11,
 '40075': 12,
 '40080': 13,
 '40085': 14,
 '40090': 15,
 '40095': 16,
 '40100': 17,
 '40200': 18,
 '40205': 19,
 '40210': 20,
 '40215': 21,
 '40220': 22,
 '40225': 23,
 '40230': 24,
 '40235': 25,
 '40300': 26,
 '40305': 27,
 '40310': 28,
 '40315': 29,
 '40316': 30,
 '40320': 31,
 '40325': 32,
 '40330': 33,
 '40335': 34,
 '40340': 35,
 '40345': 36,
 '40350': 37,
 '40355': 38,
 '40360': 39,
 '40365': 40,
 '40370': 41,
 '40375': 42,
 '40380': 43,
 '40385': 44,
 '40390': 45,
 '40395': 46,
 '40400': 47,
 '40405': 48,
 '40410': 49,
 '40415': 50,
 '40420': 51,
 '40425': 52,
 '40430': 53,
 '40435': 54,
 '40440': 55,
 '40445': 56,
 '40450': 57,
 '40455': 58,
 '40460': 59,
 '40465': 60,
 '40470': 61,
 '40475': 62,
 '40480': 63,
 '40490': 64,
 '40495': 65,
 '40500': 66,
 '40501': 67,
 '40505': 68,
 '40510': 69,
 '40520': 70,
 '40530': 71,
 '

In [18]:
sheets_dict = pd.read_excel("source/edital_pav_rod_sem_mob.xlsx", sheet_name=None)

all_sheets = []
for name, sheet in sheets_dict.items():
    sheet["sheet"] = name
    sheet = sheet.rename(columns=lambda x: x.split("\n")[-1])
    all_sheets.append(sheet)

full_table = pd.concat(all_sheets)
full_table.reset_index(inplace=True, drop=True)

In [19]:
full_table["PROJETO BÁSICO DE ENGENHARIA"] = full_table[
    "PROJETO BÁSICO DE ENGENHARIA"
].astype(str)

In [20]:
df = full_table[
    (~full_table["PROJETO BÁSICO DE ENGENHARIA"].str.contains("TOTAL"))
    & (full_table["PROJETO BÁSICO DE ENGENHARIA"].str.len() == 5)
]

In [21]:
projetos = {}
for s in df["sheet"].unique():
    writer = PdfWriter()
    servicos = df[(df["sheet"] == s)]["PROJETO BÁSICO DE ENGENHARIA"].to_list()
    pages_pdf = []
    print(servicos)
    for servico in servicos:
        if servico in global_pages.keys():
            pdf_page = global_pages.get(servico, servico)
            # print(pdf_page)
            print(pdf_page)
            pages_pdf.append(global_pages.get(servico, servico))
            writer.add_page(reader.pages[pdf_page])
            print(reader.pages[pdf_page])
        # print(global_pages.get(servico, servico))
    print(pages_pdf)
    projetos.update({s: pages_pdf})
    writer.write(f"target/composicoes/{s}.pdf")
print(projetos)

['40005', '40015', '40020', '40025', '40050', '40055', '40065', '40075', '40080', '40085', '40095', '40300', '40305', '40310', '40335', '40316', '40320', '40350', '40316', '40320', '40380', '40385', '40405', '40435', '40450', '40460', '41290', '47023', '45430', '45435', '45445', '45450', '45455', '45460', '45465', '45490', '45500', '45510', '45520', '45495', '45505', '45515', '45525', '45530', '45535', '45585', '45580', '45590', '45605', '40055', '42810', '42805', '42800', '45585', '42830', '45555', '45550', '41255', '41200', '41210', '41290', '41270', '40805', '40885', '40310', '42820', '40425', '40015', '40020', '40100', '40810', '40820', '40830', '40845', '40855', '45020', '45025', '45030', '45052', '45140', '45145', '45155', '45160', '45205', '45210', '45240', '45245', '45020', '45025', '45030', '45052', '45140', '45145', '45155', '45160', '45205', '45210', '45240', '45245', '45020', '45025', '45030', '45052', '45140', '45145', '45155', '45160', '45205', '45210', '45240', '45245', 

In [22]:
# for ws in wb.worksheets:
#     print(ws)
#     for row in range(0, ws.max_row):
#         for col in ws.iter_cols(0, 0):
#             print(col[row].value)